###### Imports

In [96]:
import pandas as pd
import numpy as np
import requests

import urllib.request
import urllib
pd.set_option('display.max_columns', None)

###### Functions for Gathering Data  
This code is from the Bureau of Labor Statistics sample functions Python 3.x Example: https://data.bls.gov/cew/doc/access/data_access_examples.htm#PYTHON

Which is built off of the open access CSV data slices (https://data.bls.gov/cew/doc/access/csv_data_slices.htm). This is a unique way to access QCEW data as it is only partially housed on the main BLS API.

In [152]:
# *******************************************************************************
# qcewCreateDataRows : This function takes a raw csv string and splits it into
# a two-dimensional array containing the data and the header row of the csv file
# a try/except block is used to handle for both binary and char encoding
def qcewCreateDataRows(csv):
    dataRows = []
    try: dataLines = csv.decode().split('\r\n')
    except er: dataLines = csv.split('\r\n');
    for row in dataLines:
        dataRows.append(row.split(','))
    return dataRows
# *******************************************************************************


# *******************************************************************************
# qcewGetAreaData : This function takes a year, quarter, and area argument and
# returns an array containing the associated area data. Use 'a' for annual
# averages. 
# For all area codes and titles see:
# http://www.bls.gov/cew/doc/titles/area/area_titles.htm
#
def qcewGetAreaData(year,qtr,area):
    urlPath = "http://data.bls.gov/cew/data/api/[YEAR]/[QTR]/area/[AREA].csv"
    urlPath = urlPath.replace("[YEAR]",year)
    urlPath = urlPath.replace("[QTR]",qtr.lower())
    urlPath = urlPath.replace("[AREA]",area.upper())
    httpStream = urllib.request.urlopen(urlPath)
    csv = httpStream.read()
    httpStream.close()
    return qcewCreateDataRows(csv)
# *******************************************************************************


# *******************************************************************************
# qcewGetIndustryData : This function takes a year, quarter, and industry code
# and returns an array containing the associated industry data. Use 'a' for 
# annual averages. Some industry codes contain hyphens. The CSV files use
# underscores instead of hyphens. So 31-33 becomes 31_33. 
# For all industry codes and titles see:
# http://www.bls.gov/cew/doc/titles/industry/industry_titles.htm
#
def qcewGetIndustryData(year,qtr,industry):
    urlPath = "http://data.bls.gov/cew/data/api/[YEAR]/[QTR]/industry/[IND].csv"
    urlPath = urlPath.replace("[YEAR]",year)
    urlPath = urlPath.replace("[QTR]",qtr.lower())
    urlPath = urlPath.replace("[IND]",industry)
    httpStream = urllib.request.urlopen(urlPath)
    csv = httpStream.read()
    httpStream.close()
    return qcewCreateDataRows(csv)
# *******************************************************************************
# *******************************************************************************
# qcewGetSizeData : This function takes a year and establishment size class code
# and returns an array containing the associated size data. Size data
# is only available for the first quarter of each year.
# For all establishment size classes and titles see:
# http://www.bls.gov/cew/doc/titles/size/size_titles.htm
#
def qcewGetSizeData(year,size):
    urlPath = "http://data.bls.gov/cew/data/api/[YEAR]/1/size/[SIZE].csv"
    urlPath = urlPath.replace("[YEAR]",year)
    urlPath = urlPath.replace("[SIZE]",size)
    httpStream = urllib.request.urlopen(urlPath)
    csv = httpStream.read()
    httpStream.close()
    return qcewCreateDataRows(csv)

# The Ask

There are a number of things we need from the QCEW in order to fulfill the Good Jobs Challenge data request.  
+ Job growth, number and percent, in the 13 county region for the past 5 years  
+ Current/last 5 years number of employees in the region to compare to the ACS 5 Year Labor Force  
+ Current top industries in the region in terms of establishment sizes, total wages, LQ (how will we put that together?) and employment  

## First we're going to look for the big job number: total employment for the last 5 years for which annual numbers are released (Q4 2021 doesn't come out until May 2022).

##### Make the lists and dictionaries that we'll want to pass into the functions.

In [3]:
#make a list of the county fips in the GNRC/WFE region
GNRC = ['47021', #Cheatham County
         '47037', #Davidson County
         '47043', #Dickson County
         '47083', #Houston County
         '47085', #Humphreys County
         '47125', #Montgomery County
         '47147', #Robertson County
         '47149', #Rutherford County
         '47161', #Stewart County
         '47165', #Sumner County
         '47169', #Trousdale County
         '47187', #Wilson County
         '47189' #Wilson County
         ]

In [4]:
#make a dictionary for frequently used regions
region = {'GNRC':['47021', #Cheatham County
         '47037', #Davidson County
         '47043', #Dickson County
         '47083', #Houston County
         '47085', #Humphreys County
         '47125', #Montgomery County
         '47147', #Robertson County
         '47149', #Rutherford County
         '47161', #Stewart County
         '47165', #Sumner County
         '47169', #Trousdale County
         '47187', #Wilson County
         '47189' #Wilson County
         ], 'MPO': ['47037', #Davidson County
                    '47119', #Maury County
                    '47147', #Robertson County
                    '47149', #Rutherford County
                    '47165', #Sumner County
                    '47187', #Wilson County
                    '47189' #Wilson County
                   ]}

In [5]:
#make a dictionary for ownership codes
ownership = {'allcovered':'0','private':'5','internationalgovt':'4','localgovt':'3','stategovt':'2','fedgovt':'1','totalgovt':'8','totalui':'9'}

In [6]:
#make a list of years
fiveyr = ['2015', '2016', '2017', '2018', '2019', '2020']

In [176]:
#make a dictionary of industry levels and lists if you need to start there, note 10s are out they're their own thing
industry2 = {'two':['11','21','22','23','42','51','52','53','54','55','56','61','62','71','72','81','92','99'], 
            'threedig':['101','102','111','112','113','114','115','211','212','213','221','236','237','238','311','312',
                        '313','314','315','316','321','322','323','324','325','326','327','331','332','333','334','335',
                        '336','337','339','423','424','425','441','442','443','444','445','446','447','448','451','452',
                        '453','454','481','482','483','484','485','486','487','488','491','492','493','511','512','515',
                        '516','517','518','519','521','522','523','524','525','531','532','533','541','551','561','562',
                        '611','621','622','623','624','711','712','713','721','722','811','812','813','814','921','922',
                        '923','924','925','926','927','928','999'], 
             'fourdig':['1111','1112','1113','1114','1119','1121','1122','1123','1124','1125','1129','1131','1132','1133','1141','1142', 
                        '1151','1152','1153','2111','2121','2122','2123','2131','2211','2212','2213','2361','2362','2371',
                        '2372','2373','2379','2381','2382','2383','2389','3111','3112','3113','3114','3115','3116','3117',
                        '3118','3119','3121','3122','3131','3132','3133','3141','3149','3151','3152','3159','3161','3162',
                        '3169','3211','3212','3219','3221','3222','3231','3241','3251','3252','3253','3254','3255','3256',
                        '3259','3261','3262','3271','3272','3273','3274','3279','3311','3312','3313','3314','3315','3321',
                        '3322','3323','3324','3325','3326','3327','3328','3329','3331','3332','3333','3334','3335','3336',
                        '3339','3341','3342','3343','3344','3345','3346','3351','3352','3353','3359','3361','3362','3363',
                        '3364','3365','3366','3369','3371','3372','3379','3391','3399','4231','4232','4233','4234','4235',
                        '4236','4237','4238','4239','4241','4242','4243','4244','4245','4246','4247','4248','4249','4251',
                        '4411','4412','4413','4421','4422','4431','4441','4442','4451','4452','4453','4461','4471','4481',
                        '4482','4483','4511','4512','4521','4522','4523','4529','4531','4532','4533','4539','4541','4542',
                        '4543','4811','4812','4821','4831','4832','4841','4842','4851','4852','4853','4854','4855','4859',
                        '4861','4862','4869','4871','4872','4879','4881','4882','4883','4884','4885','4889','4911','4921',
                        '4922','4931','5111','5112','5121','5122','5151','5152','5161','5171','5172','5173','5174','5175',
                        '5179','5181','5182','5191','5211','5221','5222','5223','5231','5232','5239','5241','5242','5251',
                        '5259','5311','5312','5313','5321','5322','5323','5324','5331','5411','5412','5413','5414','5415',
                        '5416','5417','5418','5419','5511','5611','5612','5613','5614','5615','5616','5617','5619','5621',
                        '5622','5629','6111','6112','6113','6114','6115','6116','6117','6211','6212','6213','6214','6215',
                        '6216','6219','6221','6222','6223','6231','6232','6233','6239','6241','6242','6243','6244','7111',
                        '7112','7113','7114','7115','7121','7131','7132','7139','7211','7212','7213','7221','7222','7223',
                        '7224','7225','8111','8112','8113','8114','8121','8122','8123','8129','8131','8132','8133','8134',
                        '8139','8141','9211','9221','9231','9241','9251','9261','9271','9281','9999']}
twodigit = ['11','21','22','23','42','51','52','53','54','55','56','61','62','71','72','81','92','99']

In [7]:
#make a list of quarters
qtrs = ['1', '2', '3', '4']

##### I'm going to use the get industry data function because we'll have less work on the back end having just pulled NAICS 10: All Industries

In [62]:
#use the get industry data to get the 10 digit and then chop it down by loc'in on into the ownership code and the region
appended_data = []
for yr in fiveyr:
    J = qcewGetIndustryData('{}'.format(yr), 'A', '10')
    J = pd.DataFrame(J)
    J.columns = J.iloc[0] #convert first row of return array to headers
    J = J[1:] #establish that the data is all of the rest of the rows in the return array
    J.columns = [i.replace('"', '') for i in J.columns] # cleaning data (it comes with a bunch of extra quotations)
    J = J.replace({'"':''}, regex=True) # cleaning data step 2
    appended_data.append(J)
appended_data = pd.concat(appended_data)
J = appended_data.loc[(appended_data['own_code'] =='0') & appended_data['area_fips'].isin(region['GNRC'])]
J = J.reset_index()

In [123]:
data = J

In [124]:
data.head()

,index,area_fips,own_code,industry_code,agglvl_code,size_code,year,qtr,disclosure_code,annual_avg_estabs,annual_avg_emplvl,total_annual_wages,taxable_annual_wages,annual_contributions,annual_avg_wkly_wage,avg_annual_pay,lq_disclosure_code,lq_annual_avg_estabs,lq_annual_avg_emplvl,lq_total_annual_wages,lq_taxable_annual_wages,lq_annual_contributions,lq_annual_avg_wkly_wage,lq_avg_annual_pay,oty_disclosure_code,oty_annual_avg_estabs_chg,oty_annual_avg_estabs_pct_chg,oty_annual_avg_emplvl_chg,oty_annual_avg_emplvl_pct_chg,oty_total_annual_wages_chg,oty_total_annual_wages_pct_chg,oty_taxable_annual_wages_chg,oty_taxable_annual_wages_pct_chg,oty_annual_contributions_chg,oty_annual_contributions_pct_chg,oty_annual_avg_wkly_wage_chg,oty_annual_avg_wkly_wage_pct_chg,oty_avg_annual_pay_chg,oty_avg_annual_pay_pct_chg
0,12580,47021,0,10,70,0,2015,A,,608,7995,314239281,69279906,1047498,756,39302,,1.00,1.00,1.00,1.00,1.00,1.00,1.00,,36,6.3,356,4.7,17085409,5.7,3395771,5.2,-29276,-2.7,8,1.1,402,1.0
1,12620,47037,0,10,70,0,2015,A,,20533,454563,25510537120,3990184394,48506015,1079,56121,,1.00,1.00,1.00,1.00,1.00,1.00,1.00,,909,4.6,-268,-0.1,1629224134,6.8,191558836,5.0,-920263,-1.9,69,6.8,3615,6.9
2,12635,47043,0,10,70,0,2015,A,,949,16108,586979491,142897504,2201717,701,36441,,1.00,1.00,1.00,1.00,1.00,1.00,1.00,,33,3.6,675,4.4,45120838,8.3,7542569,5.6,-298025,-11.9,26,3.9,1331,3.8
3,12735,47083,0,10,70,0,2015,A,,113,1497,45118120,11797443,190158,580,30144,,1.00,1.00,1.00,1.00,1.00,1.00,1.00,,-1,-0.9,-29,-1.9,1495316,3.4,-1117,0.0,-61656,-24.5,30,5.5,1555,5.4
4,12740,47085,0,10,70,0,2015,A,,335,5711,267779733,48551385,956926,902,46887,,1.00,1.00,1.00,1.00,1.00,1.00,1.00,,14,4.4,198,3.6,14097273,5.6,3323545,7.3,-118175,-11.0,17,1.9,875,1.9


Let's take a minute to look at the value counts in the different columns, there is some chance for double counting here so it would be smart to make sure we're not and this will be a good indication.

In [125]:
#loop through a list of all columns in the df and print the unique value counts
cols = data.columns
for col in cols:
    print(data[col].value_counts())

12719    3
12824    3
12879    3
12619    3
12604    3
12564    3
12884    3
12914    3
12924    3
12934    3
12979    3
12984    3
12724    3
12573    1
12987    1
12982    1
12937    1
12927    1
12580    1
12887    1
12613    1
12628    1
12728    1
12733    1
12833    1
12888    1
12893    1
12923    1
12933    1
12943    1
12988    1
12917    1
12567    1
12882    1
12940    1
12635    1
12735    1
12740    1
12840    1
12895    1
12900    1
12930    1
12950    1
12827    1
12995    1
13000    1
12620    1
12607    1
12622    1
12722    1
12727    1
12993    1
Name: index, dtype: int64
47021    6
47037    6
47043    6
47083    6
47085    6
47125    6
47147    6
47149    6
47161    6
47165    6
47169    6
47187    6
47189    6
Name: area_fips, dtype: int64
0    78
Name: own_code, dtype: int64
10    78
Name: industry_code, dtype: int64
70    78
Name: agglvl_code, dtype: int64
0    78
Name: size_code, dtype: int64
2015    13
2016    13
2017    13
2018    13
2019    13
2020    13
Name

Observations: for *78 rows* 
+ 6 years of data coincides with 6 instances of each FIPS code  **keep*  
+ the ownership code, 0, is repeated 78 times  **drop**  
+ the industry code, 10, all industries, is repeated 78 times  **drop**  
+ the aggregation level code, 70, is repeated 78x https://data.bls.gov/cew/doc/titles/agglevel/agglevel_titles.htm... this is a good indicator we're dealing with the correct data because this refers to the combination of geography, ownership, industry, and establishment size. We're looking at 70: County, Total Covered  **drop** 
+ size code, 0, all establishment sizes, cool  **drop** 
+ year, 13 instances of all 6 years, check  **keep**
+ quarter, 78x annual  **drop**
+ disclosure code, blank, okay  **drop**
+ annual average establishments, variety, cool  **keep**  
+ annual average employment, variety, check  **keep**  
+ total annual wages, variety, cool  **keep**  
+ taxable annual wages, good  **drop**  
You could keep going but I feel comfortable our data is sound now, so we're just going to screen through and take what we want.

This would be: year, avg annual employment

In [126]:
#breaking point in naming conventions so I don't have to keep running the API pull
data = data[['year','annual_avg_emplvl']].reset_index(drop=True)

In [127]:
data.head()

,year,annual_avg_emplvl
0,2015,7995
1,2015,454563
2,2015,16108
3,2015,1497
4,2015,5711


Now we're wanting to find the total job growth for the past 5 years, we really just need to sum the total employment for the first and last years and find the total and percent change. To do this, change the annual employment to integer.

In [128]:
#I'm using a .apply.. trying to get used to this because it's an easy way to do it to more than one column at once or to pass a list of columns in
data['annual_avg_emplvl']= data['annual_avg_emplvl'].apply(pd.to_numeric)

In [129]:
#if you don't reset the index the year becomes the index, unsure why.. but since we're transposing it that's kind of nice because it becomes the column header
data = data.groupby(['year']).sum()

In [130]:
data = data.transpose()

In [131]:
data

year,2015,2016,2017,2018,2019,2020
annual_avg_emplvl,880506,914633,944765,975365,1008240,963477


In [132]:
#calculating for year over year but we really just need 15-20
data['perc15_16'] = round(((data['2016']-data['2015'])/data['2015'])*100, 2)
data['perc16_17'] = round(((data['2017']-data['2016'])/data['2016'])*100, 2)
data['perc17_18'] = round(((data['2018']-data['2017'])/data['2017'])*100, 2)
data['perc18_19'] = round(((data['2019']-data['2018'])/data['2018'])*100, 2)
data['perc19_20'] = round(((data['2020']-data['2019'])/data['2019'])*100, 2)
data['perc15_20'] = round(((data['2020']-data['2015'])/data['2015'])*100,2 )
data['15_16'] = data['2016'] - data['2015']
data['16_17'] = data['2017'] - data['2016']
data['17_18'] = data['2018'] - data['2017']
data['18_19'] = data['2019'] - data['2018']
data['19_20'] = data['2020'] - data['2019']
data['15_20'] = data['2020'] - data['2015']

In [133]:
data.head()

year,2015,2016,2017,2018,2019,2020,perc15_16,perc16_17,perc17_18,perc18_19,perc19_20,perc15_20,15_16,16_17,17_18,18_19,19_20,15_20
annual_avg_emplvl,880506,914633,944765,975365,1008240,963477,3.88,3.29,3.24,3.37,-4.44,9.42,34127,30132,30600,32875,-44763,82971


So, we see that over the last 5 years between 2015 and 2020 the total number of jobs in the 13 county region grew by `9.42%`, or `82,971` jobs.

## Now we're going to look at the current and average of the last 5 years of employees in the region to compare to the ACS labor force estimate, because I'm curious.

We already got this information above, we'll just have to start from J and modify. We don't have to do the same EDA as we already did above.

In [114]:
data = J
data.head()

,index,area_fips,own_code,industry_code,agglvl_code,size_code,year,qtr,disclosure_code,annual_avg_estabs,annual_avg_emplvl,total_annual_wages,taxable_annual_wages,annual_contributions,annual_avg_wkly_wage,avg_annual_pay,lq_disclosure_code,lq_annual_avg_estabs,lq_annual_avg_emplvl,lq_total_annual_wages,lq_taxable_annual_wages,lq_annual_contributions,lq_annual_avg_wkly_wage,lq_avg_annual_pay,oty_disclosure_code,oty_annual_avg_estabs_chg,oty_annual_avg_estabs_pct_chg,oty_annual_avg_emplvl_chg,oty_annual_avg_emplvl_pct_chg,oty_total_annual_wages_chg,oty_total_annual_wages_pct_chg,oty_taxable_annual_wages_chg,oty_taxable_annual_wages_pct_chg,oty_annual_contributions_chg,oty_annual_contributions_pct_chg,oty_annual_avg_wkly_wage_chg,oty_annual_avg_wkly_wage_pct_chg,oty_avg_annual_pay_chg,oty_avg_annual_pay_pct_chg
0,12580,47021,0,10,70,0,2015,A,,608,7995,314239281,69279906,1047498,756,39302,,1.00,1.00,1.00,1.00,1.00,1.00,1.00,,36,6.3,356,4.7,17085409,5.7,3395771,5.2,-29276,-2.7,8,1.1,402,1.0
1,12620,47037,0,10,70,0,2015,A,,20533,454563,25510537120,3990184394,48506015,1079,56121,,1.00,1.00,1.00,1.00,1.00,1.00,1.00,,909,4.6,-268,-0.1,1629224134,6.8,191558836,5.0,-920263,-1.9,69,6.8,3615,6.9
2,12635,47043,0,10,70,0,2015,A,,949,16108,586979491,142897504,2201717,701,36441,,1.00,1.00,1.00,1.00,1.00,1.00,1.00,,33,3.6,675,4.4,45120838,8.3,7542569,5.6,-298025,-11.9,26,3.9,1331,3.8
3,12735,47083,0,10,70,0,2015,A,,113,1497,45118120,11797443,190158,580,30144,,1.00,1.00,1.00,1.00,1.00,1.00,1.00,,-1,-0.9,-29,-1.9,1495316,3.4,-1117,0.0,-61656,-24.5,30,5.5,1555,5.4
4,12740,47085,0,10,70,0,2015,A,,335,5711,267779733,48551385,956926,902,46887,,1.00,1.00,1.00,1.00,1.00,1.00,1.00,,14,4.4,198,3.6,14097273,5.6,3323545,7.3,-118175,-11.0,17,1.9,875,1.9


In [115]:
#breaking point in naming conventions so I don't have to keep running the API pull
data = data[['year','annual_avg_emplvl']].reset_index(drop=True)
data['annual_avg_emplvl']= data['annual_avg_emplvl'].apply(pd.to_numeric)
data = data.groupby(['year']).sum()
data = data.transpose()

In [116]:
data

year,2015,2016,2017,2018,2019,2020
annual_avg_emplvl,880506,914633,944765,975365,1008240,963477


So for 2019 (latest year corresponding with the ACS 5 Years) the employment number is `1,008,240`.

In [117]:
data = data.drop(columns = ['2020'])
data = data.transpose()

In [118]:
data

,annual_avg_emplvl
year,
2015,880506
2016,914633
2017,944765
2018,975365
2019,1008240


In [122]:
avg = data['annual_avg_emplvl'].mean()
print('The average annual employment between 2015 and 2019 is: ', avg)

The average annual employment between 2015 and 2019 is:  944701.8


## Now for current top industries in the region in terms of establishment sizes, total wages, LQ (how will we put that together?) and employment  

In [135]:
#let's try get area data for least memory so just for 2020, loop through 2 digit industries, loc in on the ownership code and the region

In [177]:
appended_data = []
for fips in GNRC:
    K = qcewGetAreaData('2020','A','{}'.format(fips))
    K = pd.DataFrame(K)
    K.columns = K.iloc[0] #convert first row of return array to headers
    K = K[1:] #establish that the data is all of the rest of the rows in the return array
    K.columns = [i.replace('"', '') for i in K.columns] # cleaning data (it comes with a bunch of extra quotations)
    K = K.replace({'"':''}, regex=True) # cleaning data step 2
    appended_data.append(K)
appended_data = pd.concat(appended_data)
K = appended_data.loc[(appended_data['own_code'] =='0') & appended_data['industry_code'].isin(industry2['two'])]
#J = appended_data.loc[(appended_data['own_code'] =='0') & appended_data['area_fips'].isin(region['GNRC'])]
#K = K.reset_index()

In [178]:
K

,area_fips,own_code,industry_code,agglvl_code,size_code,year,qtr,disclosure_code,annual_avg_estabs,annual_avg_emplvl,total_annual_wages,taxable_annual_wages,annual_contributions,annual_avg_wkly_wage,avg_annual_pay,lq_disclosure_code,lq_annual_avg_estabs,lq_annual_avg_emplvl,lq_total_annual_wages,lq_taxable_annual_wages,lq_annual_contributions,lq_annual_avg_wkly_wage,lq_avg_annual_pay,oty_disclosure_code,oty_annual_avg_estabs_chg,oty_annual_avg_estabs_pct_chg,oty_annual_avg_emplvl_chg,oty_annual_avg_emplvl_pct_chg,oty_total_annual_wages_chg,oty_total_annual_wages_pct_chg,oty_taxable_annual_wages_chg,oty_taxable_annual_wages_pct_chg,oty_annual_contributions_chg,oty_annual_contributions_pct_chg,oty_annual_avg_wkly_wage_chg,oty_annual_avg_wkly_wage_pct_chg,oty_avg_annual_pay_chg,oty_avg_annual_pay_pct_chg
